<a href="https://colab.research.google.com/github/Istiak-Ahmed78/Arduino-Bluetooth-Control/blob/main/leaf_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3
EPOCHS=50

In [ ]:
# from google.colab import drive
# drive.mount('/Datasets/')


In [ ]:
import zipfile
zip_path = '/content/drive/MyDrive/Datasets/PlantVillage.zip'  # Replace with your actual zip file path


In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/Datasets/tr/PlantVillage',
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 2152 files belonging to 3 classes.


In [ ]:
dataset.class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [ ]:
class_names = dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [ ]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
train_size = 0.8
validation_size = 0.1
test_size = 0.1
total_dataset_length = len(dataset)
total_dataset_length*test_size

6.800000000000001

In [ ]:
train_dataset = dataset.take(54)
rest_dataset = dataset.skip(54)
val_datas = rest_dataset.take(6)
test_datas = rest_dataset.skip(6)

In [ ]:
train_dataset[0].shape

TypeError: ignored

In [ ]:
train_ds=train_dataset.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds=val_datas.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds=test_datas.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

In [ ]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS) # Channels=3 (RGB)
n_classes = 3

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'), # Output has 3 classes. So we used n_clasees
])

model.build(input_shape=input_shape)

In [ ]:
model.compile(
optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    verbose=1,
    validation_data = val_ds,
)

Epoch 1/50


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']


In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
import numpy as np
for images_batch, labels_batch in test_ds.take(1):

    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()

    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])

    batch_prediction = model.predict(images_batch)
    print("predicted label:",class_names[np.argmax(batch_prediction[0])])

In [ ]:
model.save('/content/drive/MyDrive/Datasets/tr/mo.h5')

In [ ]:
im, l =test_ds.take(1)
l[0]

In [ ]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Datasets/tr/mo.h5')

# Show the model architecture
new_model.summary()

In [ ]:
class_names = ['Early', 'Healthy', 'Late']
class_names

['Early', 'Healthy', 'Late']

In [ ]:
def predict(model, im):
  load_input_image = tf.keras.preprocessing.image.img_to_array(im.numpy())
  raw_image = tf.expand_dims(load_input_image,0)
  predicted_data = model.predict(raw_image)
  predicted = class_names[np.argmax(predicted_data[0])]
  return predicted

In [ ]:
test_image_add = '/content/drive/MyDrive/share/0acdc2b2-0dde-4073-8542-6fca275ab974___RS_LB 4857.JPG'

In [ ]:
from PIL import Image
img = Image.open(test_image_add)

In [ ]:
import numpy as np
load_input_image = tf.keras.preprocessing.image.img_to_array(img)
raw_image = tf.expand_dims(load_input_image,0)
predicted_data = new_model.predict(raw_image)
predicted = class_names[np.argmax(predicted_data[0])]
predicted

1/1 [==============================] - 0s 18ms/step


'Potato___Late_blight'

In [ ]:
raw_image.shape

TensorShape([1, 256, 256, 3])

In [ ]:
import numpy as np
for im, l in test_datas.take(1):
  predicted = predict(new_model,im[0])
  print(predicted)

1/1 [==============================] - 0s 21ms/step
Potato___Early_blight


In [ ]:
new_model.evaluate(test_datas)

8/8 [==============================] - 5s 70ms/step - loss: 0.1382 - accuracy: 0.9828


[0.13824713230133057, 0.982758641242981]

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/share/peddy_lead_svm.sav')
converter =tf.lite.TFLiteConverter.from_keras_model( model ) # Your model's name
model = converter.convert()
file = open( '/content/drive/MyDrive/Datasets/tr/model.tflite' , 'wb' )
file.write( model )

741824

In [ ]:
import joblib
import imageio as iio
from skimage.transform import resize
import pickle

In [ ]:
def load_model():
    filename = '/content/drive/MyDrive/share/peddy_lead_svm.sav'
    loaded_model = pickle.load(open(filename, 'rb'))
    return loaded_model

In [ ]:
img = iio.imread("/content/drive/MyDrive/share/download (1).jpeg")
img_resize=resize(img,(150,150,3))
l=[img_resize.flatten()]

<ipython-input-22-09a0d3e61f22>:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = iio.imread("/content/drive/MyDrive/share/download (1).jpeg")


In [ ]:
r=load_model().predict(l)
rp=load_model().predict_proba(l)
classes = ['Leaf bright','Brown spot','Leaf smut']
print(r)
print(rp)

[0]
[[9.40443927e-01 5.92842291e-02 2.71844136e-04]]


In [ ]:
# Load the saved model
loaded_model = joblib.load('peddy_lead_svm.sav')

# Now, you can use this loaded model to make predictions
# Assuming X_test is your test data
# Replace X_test with your actual test data
predicted_labels = loaded_model.predict(X_test)

# Do something with the predicted_labels
print(predicted_labels)

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
svm_model = svm.SVC(kernel='linear', probability=True)

# Define the parameters grid to search
parameters = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    # If using 'rbf' kernel, you can also search for gamma values
    # 'gamma': [0.1, 1, 10, 100],
}

# Perform GridSearchCV to find the best combination of parameters
grid_search = GridSearchCV(estimator=svm_model, param_grid=parameters, cv=5)
grid_search.fit(X_train, y_train)  # X_train: training data, y_train: training labels

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)